In [1]:
import numpy as np
from math import sqrt
import ot
import torch
from aux_functions import a_idx, w_indices, list_pairs, fast_w_indices, gen_4mom_approx, joint_wass_dist, \
    chen_error_3step, gen_2mom_approx, make_pretty

## Demonstration of the random variable which matches the first 4 moments of Levy Area

There is also a simple version which just matches the variances conditional on the W increment

In [2]:
data = np.genfromtxt('samples/samples_4-dim.csv', dtype=float, delimiter=',')
a_true = data[:(2*16384),4:10]
W = data[:(2*16384),:4]
bsz = 2*16384
generated4mom = gen_4mom_approx(4, bsz, _W=W)
generated2mom = gen_2mom_approx(4, bsz, _W = W)
a_generated4mom = generated4mom[:, 4:10]
a_generated2mom = generated2mom[:, 4:10]

(32768, 6)


In [4]:
err = [sqrt(ot.wasserstein_1d(a_true[:,i], a_generated4mom[:, i], p=2)) for i in range(4)]
print(err)

[0.0031215251014375177, 0.0031286148594010174, 0.002799245139591539, 0.0023474843729451474]


In [5]:
err = [sqrt(ot.wasserstein_1d(a_true[:,i], a_generated2mom[:, i], p=2)) for i in range(4)]
print(err)

[0.020255162715319474, 0.020892199738661764, 0.019990485264570586, 0.020879712144683007]


In [ ]:
joint_err = joint_wass_dist(a_true[:16384], a_true[16384:(2*16384)])
print(joint_err)

In [ ]:
joint_err = joint_wass_dist(a_true[:10000], a_generated2mom[:10000])
print(joint_err)

## A utility for calculating the empirical fourth moments of a set of samples

In [8]:
def four_combos(n: int):
    lst = []
    for i in range(n):
        for j in range(i,n):
            for k in range(j,n):
                for l in range(k,n):
                    lst.append((i,j,k,l))
    return lst

def fourth_moments(input_samples: np.ndarray):
    dim = input_samples.shape[1]
    lst = four_combos(dim)
    res = []
    for i,j,k,l in lst:
        col = input_samples[:, i] * input_samples[:, j] * input_samples[:, k] * input_samples[:, l]
        res.append(col.mean())
    return res


w2 = np.concatenate((W,W,W,W), axis=0)
print(w2.shape)
combo_list = four_combos(3)
moms = fourth_moments(generated4mom)
moms2 = fourth_moments(a_true)
for i in range(len(combo_list)):
    print(f"combo: {combo_list[i]}, 4_match_RV moment: {moms[i] :.7f}, samples moment: {moms2[i] :.7f}")

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 0, 2), (0, 0, 1, 1), (0, 0, 1, 2), (0, 0, 2, 2), (0, 1, 1, 1), (0, 1, 1, 2), (0, 1, 2, 2), (0, 2, 2, 2), (1, 1, 1, 1), (1, 1, 1, 2), (1, 1, 2, 2), (1, 2, 2, 2), (2, 2, 2, 2)]


In [10]:
w2 = np.concatenate((W,W,W,W), axis=0)
print(w2.shape)

(262144, 3)


In [11]:
generated4mom = gen_4mom_approx(3, 4 * bsz, _W=w2)
moms = fourth_moments(generated4mom)
for i in range(len(combo_list)):
    print(f"combo: {combo_list[i]}, 4_match_RV moment: {moms[i] :.7f}, samples moment: {moms2[i] :.7f}")

combo: (0, 0, 0, 0), 4_match_RV moment: 0.1350575, samples moment: 0.1357276
combo: (0, 0, 0, 1), 4_match_RV moment: 0.0336348, samples moment: 0.0357896
combo: (0, 0, 0, 2), 4_match_RV moment: 0.0655758, samples moment: 0.0649001
combo: (0, 0, 1, 1), 4_match_RV moment: 0.0722153, samples moment: 0.0728633
combo: (0, 0, 1, 2), 4_match_RV moment: 0.0007437, samples moment: 0.0013768
combo: (0, 0, 2, 2), 4_match_RV moment: 0.0717766, samples moment: 0.0717662
combo: (0, 1, 1, 1), 4_match_RV moment: 0.0481259, samples moment: 0.0485544
combo: (0, 1, 1, 2), 4_match_RV moment: 0.0280197, samples moment: 0.0279942
combo: (0, 1, 2, 2), 4_match_RV moment: 0.0045032, samples moment: 0.0053913
combo: (0, 2, 2, 2), 4_match_RV moment: 0.0753795, samples moment: 0.0750950
combo: (1, 1, 1, 1), 4_match_RV moment: 0.2946441, samples moment: 0.2966960
combo: (1, 1, 1, 2), 4_match_RV moment: -0.0395968, samples moment: -0.0387191
combo: (1, 1, 2, 2), 4_match_RV moment: 0.0827522, samples moment: 0.08177